# Testing corrected y_pred

In [251]:
%load_ext autoreload
%autoreload 2

import utils
import run

import numpy as np
import torch
import matplotlib.pyplot as plt

# test config
config = run.make_config('configs/horizonwithhead_default.yaml') # window 100, horizon 100
torch_dict = torch.load('models/horizonwithheadlstm_default_17-11-2023_13-22-36_e3800.p', map_location=torch.device('cpu'))
config['model'].load_state_dict(torch_dict['model_state_dict'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
using cpu


<All keys matched successfully>

In [328]:
def visualize_differences(config, i=None, j=None, ax=None):
    model = config['model']
    if i is None:
        i = np.random.randint(len(config['train_datasets']))
    if j is None:
        j = np.random.randint(len(config['train_datasets'][i]))
    x, y, m = config['train_datasets'][i][j]
    m = m.bool().unsqueeze(-1).expand(-1, y.shape[-1]).detach().numpy()

    model.eval()
    
    y_pred, _ = model(x)
    y_pred = y_pred[:, -2:].detach().numpy()
    y_pred *= m
    y *= m
    y = y.detach().numpy()
    
    y, y_pred, m = y[-config['horizon']:], y_pred[-config['horizon']:], m[-config['horizon']:]
    
    y_traj = y.cumsum(axis=0)
    y_pred_traj = y_pred.cumsum(axis=0)

    y_corrected = utils.get_corrected_y_for_delta_p(torch.from_numpy(y), y_pred, m)

    y_traj_x = [0, *y_traj[:, 0]]
    y_traj_y = [0, *y_traj[:, 1]]

    y_pred_traj_x = [0, *y_pred_traj[:, 0]]
    y_pred_traj_y = [0, *y_pred_traj[:, 1]]

    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(y_traj_x, y_traj_y, "-o", label='true')
    ax.plot(y_pred_traj_x, y_pred_traj_y, "-o", label='predicted')
    for k in range(y_corrected.shape[0]):
        ax.plot([y_pred_traj_x[k], y_pred_traj_x[k] + y_corrected[k, 0]], [y_pred_traj_y[k], y_pred_traj_y[k] + y_corrected[k, 1]])
    ax.legend()
    ax.set_title(f'i = {i}, j={j}')

    # set lim
    itx, atx = min(y_traj_x), max(y_traj_x)
    ity, aty = min(y_traj_y), max(y_traj_y)
    if abs(atx - itx) > abs(aty - ity):
        diff = abs(atx - itx - aty + ity) // 2
        aty += diff
        ity -= diff
    else:
        diff = abs(aty - ity - atx + itx) // 2
        atx += diff
        itx -= diff
    ax.set_xlim(itx, atx)
    ax.set_ylim(ity, aty)
    
    return y, y_pred, y_traj, y_pred_traj, y_corrected

y, y_pred, y_traj, y_pred_traj, y_corrected = visualize_differences(config)#, 0, 17030) #, 0, 5707)
#y, y_pred, y_traj, y_pred_traj, y_pred_corrected

AttributeError: 'HorizonWithHeadLSTM' object has no attribute 'head'